In [ ]:
import os
from pathlib import Path
from glob import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib.lines import Line2D
from matplotlib.patches import Patch

#from label_flip_revised.utils import create_dir

In [ ]:
PATH_ROOT = Path(os.getcwd()).absolute().parent
print(PATH_ROOT)

In [ ]:
path_output = os.path.join(PATH_ROOT, 'results_plot')
print(path_output)
#create_dir(path_output)

In [ ]:
df_falfa_01 = pd.read_csv(os.path.join(PATH_ROOT, 'results', 'real', 'real_falfa_nn_db.csv'))
print(df_falfa_01.shape)
df_falfa_01.head()

In [ ]:
df_falfa_01.columns

In [ ]:
datanames = np.sort(df_falfa_01['Data.Base'].unique())
print(datanames)

In [ ]:
workspace_names = [f'diva_{i:02d}' for i in range(1,6)]
workspace_names

In [ ]:
ATTACKS = ['rand_svm', 'alfa_svm', 'poison_svm', 'falfa_nn']
ATTACKS


In [ ]:
score_dict = {}
for dname in ATTACKS:
    dfs = []
    for wname in workspace_names:
        df_ = pd.read_csv(os.path.join(PATH_ROOT.parent, wname, 'results', 'real', f'real_{dname}_db.csv'))
        df_ = df_[['Data.Base', 'Rate', 'Train.Clean', 'Test.Clean', 'Train.Poison', 'Test.Poison']]
        dfs.append(df_)
    df = pd.concat(dfs, ignore_index=True).groupby(['Data.Base','Rate']).mean()
    df.to_csv(os.path.join(path_output, f'real_score_mean_{dname}.csv'))

In [ ]:
COLOR_MAP = {
    'rand_svm': 'darkgrey',
    'alfa_svm': '#2e7d32',
    'poison_svm': '#1976d2',
    'falfa_nn': '#d32f2f',
    #  'Train.Poison': '#1b1b1b', 
    #  'Test.Poison': '#1b1b1b',
}

my_palette = sns.color_palette(COLOR_MAP.values())
my_palette


In [ ]:
TITLES = [
    'Abalone',
    'Australian',
    'Banknote',
    'Breastcancer',
    'CMC',
    'HTRU2',
    'Phoneme',
    'Ringnorm',
    'Texture',
    'Yeast'
]

In [ ]:
datanames = ['abalone_subset_std', 'australian_std', 'banknote_std', 'breastcancer_std',
 'cmc_std', 'htru2_subset_std', 'phoneme_subset_std', 'ringnorm_subset_std',
 'texture_subset_std', 'yeast_subset_std']

In [ ]:
# Save results
df_falfa = pd.read_csv(os.path.join(path_output, f'real_score_mean_falfa_nn.csv'))
df_rand = pd.read_csv(os.path.join(path_output, f'real_score_mean_rand_svm.csv'))
df_pois = pd.read_csv(os.path.join(path_output, f'real_score_mean_poison_svm.csv'))
df_alfa = pd.read_csv(os.path.join(path_output, f'real_score_mean_alfa_svm.csv'))

df_falfa.head()

In [ ]:
def draw_subplot(dfs, dataname, attacks, ax, linewidth=2.5, markers=True):
    df_table = pd.DataFrame()
    for df_, att in zip(dfs, attacks):
        df_ = df_ * 100
        df_['Attack'] = att
        df_ = df_[['Attack', 'Rate', 'Test.Poison', 'Train.Poison']]
        df_ = df_.melt(id_vars=['Attack', 'Rate'], var_name='Set', value_name='Accuracy')
        df_table = pd.concat([df_table, df_], ignore_index=True)
        
    line_style = ['Train' if val == 'Train.Poison' else 'Test' for val in df_table['Set']]
    sns.lineplot(data=df_table,
                 x='Rate',
                 y='Accuracy',
                 hue='Attack',
                 ax=ax,
                 style=line_style,
                 legend='brief',
                 alpha=0.8,
                 palette=my_palette,
                 linewidth=linewidth,
                 #markers=markers,
                 dashes=True,
    )

    ax.set(xlim=[-1, 41], ylim=[35, 105])
    ax.set_title(dataname, fontsize=FONTSIZE+1)
    ax.set_xlabel('Poisoning Rate (%)', fontsize=FONTSIZE+1)
    ax.set_ylabel('Accuracy (%)', fontsize=FONTSIZE+1)
    ax.set_xticks(np.linspace(0, 40, num=5, endpoint=True), fontsize=FONTSIZE-1)
    ax.set_yticks(np.linspace(40, 100, num=7, endpoint=True), fontsize=FONTSIZE-1)
    ax.get_legend().remove()

In [ ]:
FONTSIZE = 13
FIGSIZE = (16, 5.5)
LINE_WIDTH = 3

plt.rcParams["font.size"] = FONTSIZE
fig, axs = plt.subplots(2, 5, sharey=True, sharex=True, figsize=FIGSIZE)
_axs = []
for i in range(2):
    for j in range(5):
        _axs.append(axs[i, j])
for dataname, title, ax in zip(datanames, TITLES, _axs):
    dfs = [
        df_rand[df_rand['Data.Base'] == dataname],
        df_alfa[df_alfa['Data.Base'] == dataname],
        df_pois[df_pois['Data.Base'] == dataname],
        df_falfa[df_falfa['Data.Base'] == dataname],
    ]
    draw_subplot(dfs, title, ATTACKS, ax, linewidth=2.5, markers=True)

# Merge lines and labels
# lines, labels = _axs[i].get_legend_handles_labels()
# label_dict = {
#     'rand_svm': 'SLN',
#     'alfa_svm': 'ALFA',
#     'poison_svm': 'PoisSVM',
#     'falfa_nn': 'FALFA',
#     'Train': 'Train',
#     'Test': 'Test',
# }

custom_lines = [
    Line2D([0], [0], color='black', lw=LINE_WIDTH, label='Training Acc.', linestyle='dashed'),
    Line2D([0], [0], color='black', lw=LINE_WIDTH, label='Test Acc.'),
    Patch(facecolor=COLOR_MAP['falfa_nn'], label='FALFA'),
    Patch(facecolor=COLOR_MAP['alfa_svm'], label='ALFA'),
    Patch(facecolor=COLOR_MAP['poison_svm'], label='PoisSVM'),
    Patch(facecolor=COLOR_MAP['rand_svm'], label='Noise (SLN)')
]

# labels_new = []
# for label in labels:
#     labels_new.append(label_dict[label])
fig.legend(handles=custom_lines, loc='lower left', bbox_to_anchor=(0, 1, 1, 0), mode="expand", ncol=6)
# fig.legend(lines, labels_new, bbox_to_anchor=(0, 1, 1, 0), loc="lower left", mode="expand", ncol=6)

plt.tight_layout(pad=0.3)
path_fig_out = os.path.join(path_output, 'flfa_acc_all.pdf', )
plt.savefig(path_fig_out, dpi=300, bbox_inches='tight')
print(f'Save plot to {path_fig_out}')